In [189]:
import folium
import requests
import pandas as pd
import numpy as np

In [190]:
# ref : https://github.com/infoBMKG/data-gempabumi

autogempa = "https://data.bmkg.go.id/DataMKG/TEWS/autogempa.json"
gempabumiterkini = "https://data.bmkg.go.id/DataMKG/TEWS/gempaterkini.json"
gempabumidirasakan = "https://data.bmkg.go.id/DataMKG/TEWS/gempadirasakan.json"

# Function

In [191]:
class StoreBMKG():
    def __init__(self, data: str):
        self.__get_bmkg(data)

    def __auto_earthquake(self, filename, data: dict):
        data = pd.DataFrame.from_dict(data, orient='index').T
        data.to_csv('dataset/'+filename+'.csv', index=True)

    # need to improve table for more tidy csv from felt to new
    def __felt_earthquake(self, filename, data: pd.DataFrame):
        columns = ['Tanggal','Jam','DateTime','Coordinates','Lintang','Bujur','Magnitude','Kedalaman','Wilayah', 'Dirasakan']

        rows = []
        for d in data:
            rows.append(pd.Series(d, index=columns))
        
        datas = pd.DataFrame(rows, columns=columns)
        datas.to_csv('dataset/'+filename+'.csv', index=True)
            
    def __new_earthquake(self, filename, data: pd.DataFrame):
        columns = ['Tanggal','Jam','DateTime','Coordinates','Lintang','Bujur','Magnitude','Kedalaman','Wilayah', 'Potensi']

        rows = []
        for d in data:
            rows.append(pd.Series(d, index=columns))
        
        datas = pd.DataFrame(rows, columns=columns)
        datas.to_csv('dataset/'+filename+'.csv', index=True)

    def __get_bmkg(self, data:str):

        from io import StringIO
        
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 \
                    (Macintosh; Intel Mac OS X 10_11_5) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/50.0.2661.102 Safari/537.36'
            }
            
            response = requests.get(data, headers=headers)
            raw = pd.read_json(StringIO(response.content.decode('utf-8')))
            
            filename = data.split('/')[-1].split('.')[0]

            if filename == 'autogempa':
                self.__auto_earthquake(filename, raw['Infogempa']['gempa'])
            elif filename == 'gempadirasakan':
                self.__felt_earthquake(filename, raw['Infogempa']['gempa'])
            elif filename == 'gempaterkini':
                self.__new_earthquake(filename, raw['Infogempa']['gempa'])
                
        except requests.exceptions.HTTPError as errh:
            print(repr(errh))
        except requests.exceptions.ConnectionError as errc:
            print(repr(errc))
        except requests.exceptions.Timeout as errt:
            print(repr(errt))
        except requests.exceptions.RequestException as err:
            print(repr(err))

# Storing Data into CSV

In [192]:
listDataBMKG = [autogempa, gempabumidirasakan, gempabumiterkini]
for d in listDataBMKG:
    StoreBMKG(d)

# Data BMKG Gempa

## Show Table

In [193]:
dataAutoGempa = pd.read_csv('dataset/autogempa.csv')

dataAutoGempa.drop(dataAutoGempa.columns[
    dataAutoGempa.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

display(dataAutoGempa)

,Tanggal,Jam,DateTime,Coordinates,Lintang,Bujur,Magnitude,Kedalaman,Wilayah,Potensi,Dirasakan,Shakemap
0,26 Mar 2024,00:36:36 WIB,2024-03-25T17:36:36+00:00,"0.83,125.40",0.83 LU,125.40 BT,4.8,40 km,Pusat gempa berada di laut 71 km Tenggara Ratahan,Gempa ini dirasakan untuk diteruskan pada masy...,"II-III Bitung, II Manado, II Tondano",20240326003636.mmi.jpg


## Show Map

### Folium

In [194]:
import math

def find_distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(float(lat2)-lat1)
    dlon = math.radians(float(lon2)-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(float(lat2))) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance_bw_ori_desti = radius * c
    return distance_bw_ori_desti

In [204]:
# ref : https://www.instrovate.com/2019/07/01/tagging-two-locations-on-map-using-python-and-finding-distance-between-two-tagged-location-in-python/

from collections import namedtuple

def getArrows(locations, color='blue', size=6, n_arrows=3):

    Point = namedtuple('Point', field_names=['lat', 'lon'])
    
    point1 = Point(locations[0][0], locations[0][1])
    point2 = Point(float(locations[1][0]), float(locations[1][1]))
    
    angle = get_angle(point1, point2) - 90
    
    arrow_latitude = np.linspace(point1.lat, point2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_longitude = np.linspace(point1.lon, point2.lon, n_arrows + 2)[1:n_arrows+1]
    
    final_arrows = []
    
    for points in zip(arrow_latitude, arrow_longitude):
        final_arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=angle))
    return final_arrows

def get_angle(p1, p2):

    longitude_diff = np.radians(float(p2.lon) - p1.lon)
    
    latitude1 = np.radians(p1.lat)
    latitude2 = np.radians(float(p2.lat))
    
    x_vector = np.sin(longitude_diff) * np.cos(latitude2)
    y_vector = (np.cos(latitude1) * np.sin(latitude2) 
        - (np.sin(latitude1) * np.cos(latitude2) 
        * np.cos(longitude_diff)))
    angle = np.degrees(np.arctan2(x_vector, y_vector))
    
    if angle < 0:
        return angle + 360
    return angle

In [213]:
quake = [0, 0]
surabaya = [-7.2754417, 112.6302827]

for v in dataAutoGempa['Coordinates'].values:
    quake = v.split(',')

centerlat = (float(quake[0]) + float(quake[1])) / 2
centerlon = (surabaya[0] + surabaya[1]) / 2

map = folium.Map(location=[centerlat, centerlon], zoom_start=7)

estimated_distance = find_distance(surabaya, quake)
print(f"Estimated distance: {estimated_distance:.2f} kilometers")

html=f"""
        <span style="size: 10px; background-color: lightblue; ">
        <p> distance of surabaya and quake {estimated_distance}</p></span>
        """
iframe = folium.IFrame(html=html, width=130, height=130)
popup = folium.Popup(iframe, max_width=165)

folium.Marker(quake).add_to(map)
folium.Marker(surabaya).add_to(map)
folium.PolyLine(popup=popup, locations=[surabaya, quake], color='red').add_to(map)

arrows = getArrows(locations=[surabaya, quake], n_arrows=7)

for arrow in arrows:
    arrow.add_to(map)

map


Estimated distance: 1678.90 kilometers


# Data BMKG Gempa Dirasakan

# Data BMKG Gempa Terkini

In [197]:
DataBMKG(gempabumiterkini)